# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
!pip install scikit-learn==0.24.2

In [46]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.19.1.


In [48]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from Custom_Transformers import MessageTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ImportError: cannot import name '_safe_indexing'

In [37]:
# load data from database
engine = create_engine('sqlite:///Vasilis_db.db')
df = pd.read_sql_table('Emergency_Messages', engine)
df = df.drop(columns = ['id','original'], axis=1)

In [38]:
# drop rows with NaN messages
df = df.dropna(subset=['message'])

In [4]:
# our dataset still contains some null values
df.isnull().sum()

message                    0
genre                      0
related                   32
request                   32
offer                     32
aid_related               32
medical_help              32
medical_products          32
search_and_rescue         32
security                  32
military                  32
child_alone               32
water                     32
food                      32
shelter                   32
clothing                  32
money                     32
missing_people            32
refugees                  32
death                     32
other_aid                 32
infrastructure_related    32
transport                 32
buildings                 32
electricity               32
tools                     32
hospitals                 32
shops                     32
aid_centers               32
other_infrastructure      32
weather_related           32
floods                    32
storm                     32
fire                      32
earthquake    

In [39]:
df = df.dropna()

In [40]:
# We can observe that the 'related' category also contains double's(2) that does not make sense - we will turn this 2s to 1s
df.related.value_counts()

1.0    19879
0.0     6111
2.0      188
Name: related, dtype: int64

In [41]:
df.loc[df.related==2, 'related'] = 1

In [42]:
X = df[['message','genre']]
Y = df.drop(columns=['message','genre'], axis=1)

In [57]:
df.genre

0        direct
1        direct
2        direct
3        direct
4        direct
5        direct
6        direct
7        direct
8        direct
9        direct
10       direct
11       direct
12       direct
13       direct
14       direct
15       direct
16       direct
17       direct
18       direct
19       direct
20       direct
21       direct
22       direct
23       direct
24       direct
25       direct
26       direct
27       direct
28       direct
29       direct
          ...  
26180      news
26181      news
26182      news
26183      news
26184      news
26185      news
26186      news
26187      news
26188      news
26189      news
26190      news
26191      news
26192      news
26193      news
26194      news
26195      news
26196      news
26197      news
26198      news
26199      news
26200      news
26201      news
26202      news
26203      news
26204      news
26205      news
26206      news
26207      news
26208      news
26209      news
Name: genre, Length: 261

### 2. Write a tokenization function to process your text data

In [44]:
def tokenize(text):
    """ Function that tokenizes and lemmatizes text

    :param text:     input text to be processed(str)
    :return:         cleaned_tokens(str)
    """

    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()

    # Text normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    #tokenization
    words = word_tokenize (text)
    # lemmatization
    cleaned_tokens = [lemmatizer.lemmatize(word).strip() for word in words if word not in stop_words]

    return cleaned_tokens 

In [ ]:
#!conda install xgboost

Solving environment: | 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
     ('vect', CountVectorizer(tokenizer=tokenize)),
     ('tfidf', TfidfTransformer()),
     ('clf', RandomForestClassifier(random_state=33))
])

In [45]:
text_transformer = Pipeline([
    ('messagetransformer', MessageTransformer(tokenize)),
])
genre_transformer = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('text', text_transformer, [0]),
    ('genre', genre_transformer, [1])
])
pipeline_2 = Pipeline([
    ('preprocess', preprocessor),
    ('clf', RandomForestClassifier(random_state=33))
])

NameError: name 'ColumnTransformer' is not defined

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=33)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimators=10, n_jobs=1,
            oob_score=False, random_state=33, verbose=0, warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
preds = pipeline.predict(X_test)

In [15]:
pred_cols = y_test.columns.tolist()
for i,col in enumerate(pred_cols):
     print(col.upper()+'\n', classification_report(y_test[col].values, preds[:,i]))

RELATED
              precision    recall  f1-score   support

        0.0       0.32      0.20      0.24      1517
        1.0       0.78      0.87      0.83      5028

avg / total       0.68      0.72      0.69      6545

REQUEST
              precision    recall  f1-score   support

        0.0       0.84      0.97      0.90      5458
        1.0       0.37      0.09      0.15      1087

avg / total       0.76      0.82      0.78      6545

OFFER
              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6513
        1.0       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6545

AID_RELATED
              precision    recall  f1-score   support

        0.0       0.59      0.84      0.69      3810
        1.0       0.46      0.19      0.27      2735

avg / total       0.54      0.57      0.52      6545

MEDICAL_HELP
              precision    recall  f1-score   support

        0.0       0.92      1.00  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
    'vect__ngram_range' : ((1, 1), (1, 2)),
    'clf__n_estimators' :[50,100,200],
    'clf__max_depth'        :[10,20,None]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3)
cv.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimators=10, n_jobs=1,
            oob_score=False, random_state=33, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'clf__n_estimators': [50, 100, 200], 'clf__max_depth': [10, 20, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv_preds = cv.predict(X_test)
pred_cols = y_test.columns.tolist()
for i,col in enumerate(pred_cols):
     print(col.upper()+'\n', classification_report(y_test[col].values, cv_preds[:,i]))

RELATED
              precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      1517
        1.0       0.77      1.00      0.87      5028

avg / total       0.59      0.77      0.67      6545

REQUEST
              precision    recall  f1-score   support

        0.0       0.83      1.00      0.91      5458
        1.0       0.00      0.00      0.00      1087

avg / total       0.70      0.83      0.76      6545

OFFER
              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6513
        1.0       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6545

AID_RELATED
              precision    recall  f1-score   support

        0.0       0.58      1.00      0.74      3810
        1.0       0.00      0.00      0.00      2735

avg / total       0.34      0.58      0.43      6545

MEDICAL_HELP
              precision    recall  f1-score   support

        0.0       0.92      1.00  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [17]:
import pickle
pickle.dump(cv, open('rf_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.